Conduct RFM (recency, frequency, monetary value) analysis on political donors in the UK

In [7]:
#imports
import pandas as pd

In [23]:
#read data
df = pd.read_csv('data/Donations accepted by political parties.csv')
df = df.dropna()
df['ReceivedDate']= pd.to_datetime(df['ReceivedDate'])

/var/folders/9t/ywq9gg3s4zbgn2zw5tfskbv00000gn/T/ipykernel_21627/3644051415.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Donations accepted by political parties.csv')


ValueError: Bin edges must be unique: Index([nan, nan, nan, nan, nan, nan], dtype='float64', name='recency').
You can drop duplicate edges by setting the 'duplicates' kwarg